# Optimal Control 16.323

In [138]:
import numpy as np
from functools import partial
from itertools import product
np.set_printoptions(threshold=np.inf)
from bidict import bidict

# Lecture 3/4: Dynamic Programming and HJB

In [139]:
Nx = [50,100]
NX = Nx[0]*Nx[1]
n = len(Nx)

def flati(t):
  i,j = t
  if i < 0 or j < 0:
    return None
  return i*Nx[1]+j
def tuplei(i):
  if i > NX:
    return None
  return divmod(i,Nx[1])

In [140]:
Nt = 100
x1 = np.linspace(-5,5,Nx[0])
x2 = np.linspace(-5,5,Nx[1])
dx1 = x1[1]-x1[0]
dx2 = x2[1]-x2[0]
dx = np.array([dx1,dx2])
x0 = np.array([x1[0],x2[0]])

Nu = [41]
NU = Nu[0]
u = np.linspace(-15,15,NU)

Dt = .2
A = np.array([[3,-1],[-.5,5]])
B = np.array([[0],[1]])
def xdot(xk,uk,k):
  return A@xk + B*uk
def final_cost(xvec):
  return xvec[0]**2 + 2 * xvec[1]**2
Jstar = np.zeros((Nt,NX)) # J*_k(x_k^i) k is time index
Jstar[Nt-1,:] = [final_cost([x,y]) for (x,y) in list(product(x1,x2))]
print("J*_T")
print(np.round(Jstar[Nt-1,:].reshape(Nx)))
print(np.argmin(Jstar[Nt-1,:]))

J*_T
[[75. 73. 71. 69. 67. 65. 64. 62. 60. 58. 57. 55. 54. 52. 51. 49. 48. 47.
  45. 44. 43. 42. 40. 39. 38. 37. 36. 35. 34. 34. 33. 32. 31. 31. 30. 29.
  29. 28. 28. 27. 27. 26. 26. 26. 26. 25. 25. 25. 25. 25. 25. 25. 25. 25.
  25. 26. 26. 26. 26. 27. 27. 28. 28. 29. 29. 30. 31. 31. 32. 33. 34. 34.
  35. 36. 37. 38. 39. 40. 42. 43. 44. 45. 47. 48. 49. 51. 52. 54. 55. 57.
  58. 60. 62. 64. 65. 67. 69. 71. 73. 75.]
 [73. 71. 69. 67. 65. 63. 62. 60. 58. 56. 55. 53. 52. 50. 49. 47. 46. 45.
  43. 42. 41. 40. 38. 37. 36. 35. 34. 33. 32. 32. 31. 30. 29. 29. 28. 27.
  27. 26. 26. 25. 25. 24. 24. 24. 24. 23. 23. 23. 23. 23. 23. 23. 23. 23.
  23. 24. 24. 24. 24. 25. 25. 26. 26. 27. 27. 28. 29. 29. 30. 31. 32. 32.
  33. 34. 35. 36. 37. 38. 40. 41. 42. 43. 45. 46. 47. 49. 50. 52. 53. 55.
  56. 58. 60. 62. 63. 65. 67. 69. 71. 73.]
 [71. 69. 67. 65. 63. 61. 60. 58. 56. 55. 53. 51. 50. 48. 47. 45. 44. 43.
  41. 40. 39. 38. 37. 35. 34. 33. 32. 31. 31. 30. 29. 28. 27. 27. 26. 25.
  25. 24. 24. 23. 23.

In [148]:
def gij_k(xi_k, uij_k, k):
  return .2*(uij_k-.01)**2
ctrlmap = bidict()
for ui in u:
   ctrlmap[np.round(ui,3)] = np.round(gij_k(0,ui,0),3)



In [149]:
print(ctrlmap)

bidict({np.float64(-15.0): np.float64(45.06), np.float64(-14.25): np.float64(40.67), np.float64(-13.5): np.float64(36.504), np.float64(-12.75): np.float64(32.564), np.float64(-12.0): np.float64(28.848), np.float64(-11.25): np.float64(25.358), np.float64(-10.5): np.float64(22.092), np.float64(-9.75): np.float64(19.052), np.float64(-9.0): np.float64(16.236), np.float64(-8.25): np.float64(13.646), np.float64(-7.5): np.float64(11.28), np.float64(-6.75): np.float64(9.14), np.float64(-6.0): np.float64(7.224), np.float64(-5.25): np.float64(5.534), np.float64(-4.5): np.float64(4.068), np.float64(-3.75): np.float64(2.828), np.float64(-3.0): np.float64(1.812), np.float64(-2.25): np.float64(1.022), np.float64(-1.5): np.float64(0.456), np.float64(-0.75): np.float64(0.116), np.float64(0.0): np.float64(0.0), np.float64(0.75): np.float64(0.11), np.float64(1.5): np.float64(0.444), np.float64(2.25): np.float64(1.004), np.float64(3.0): np.float64(1.788), np.float64(3.75): np.float64(2.798), np.float64(4

In [150]:
def get_vec(ii):
   x,y = tuplei(ii)
   return np.array([x1[x],x2[y]])
def grid_interp(x_c,x_i,c_i):
  # x_i candidates Nxn where N candidates x in R^n
  # c_i is cost Nx1
  w_i = 1./np.linalg.norm(x_i-x_c,axis=1)
  print(w_i)
  print(np.dot(w_i,c_i))
  return np.dot(w_i,c_i)/np.sum(w_i)

  # demo pts
  # xc = np.array([2,3])
  # pts = np.array([[2.01,3.01],[1.66,2.53],[1.6,3.7]])
  # cpts = np.array([9.6,10.2,11.0])
  # print(grid_interp(xc,pts,cpts))

def get_nearest_points(ii):
    x,y = tuplei(ii)
    points = []
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < Nx[0] and 0 <= new_y < Nx[1]:
                points.append(flati((new_x, new_y)))
    return points

def closest_gridpoint(xi):
  return np.int32((xi-x0+dx/2)/dx)
print(flati(closest_gridpoint(np.array([.3,2.4]))))

2673


In [151]:
# all possible states X all possible controls X [x^j_k+1 in R^n, interpolation shift, cost]
k = Nt-2
Cik2jk1 = np.full((NX,NX),np.inf)
for i in range(NX): # all states
    xi = get_vec(i)
    xdot_uk = partial(xdot,xk=xi.reshape(2,1),k=k) # TODO jit
    for j in range(NU): # all controls
      cost_j = gij_k(xi,u[j],k)
      xdot_vec = xdot_uk(uk=u[j]).flatten()

      xj = xi+xdot_vec*Dt
      # print(xi,xj)
      closest_index = closest_gridpoint(xj)
      if closest_index[0] < 0 or closest_index[1] < 0 or closest_index[0] >= Nx[0] or closest_index[1] >= Nx[1]:
         continue
      transfer_index = flati(closest_index)

      nearby_indices = get_nearest_points(transfer_index)
      Cik2jk1[i,transfer_index] = cost_j
      # Cik2jk1[i,[transfer_index] + nearby_indices] = cost_j
      # TODO grid_interp(x_c=xi,x_i=[get_vec(ii) for ii in nearby_indices])


Apply DP Formulae

In [152]:
for k in reversed(range(Nt-2)):
  for i in range(NX):
    Jstar[k,i] = np.min(Cik2jk1[i,:] + Jstar[k+1,:])
  

In [154]:
Nsteps = 50
# control sequence
best_seq = np.argmin(Jstar[0,:])
print(best_seq)
print(Jstar[0,best_seq])
for i in range(int(4*NX/10.),int(NX*6./10.)):
  seq = [i]
  for k in range(1,Nsteps-1):
    target_step = np.argmin(Cik2jk1[seq[k-1],:] + Jstar[k,:])
    inverted_cost = Cik2jk1[seq[k-1],target_step]
    if inverted_cost == np.inf:
      break
    ctrl = ctrlmap.inverse[np.round(inverted_cost,3)]
    xki = get_vec(seq[-1])
    print(ctrl)
    xk1i = xki.reshape(2,1) + Dt*xdot(xki.reshape(2,1),ctrl,k)
    seq.append(flati(closest_gridpoint(xk1i.flatten())))
  seq.append(np.argmin(Jstar[-1,:]))
  # print(f"Optimal Sequence for state {i+1}: ")
  print("->".join([str(i+1) for i in seq]))

2449
0.001960000000000004
2001->2450
2002->2450
2003->2450
2004->2450
2005->2450
2006->2450
2007->2450
2008->2450
2009->2450
2010->2450
2011->2450
2012->2450
2013->2450
2014->2450
2015->2450
2016->2450
2017->2450
2018->2450
2019->2450
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2020->2450
15.0
15.0
14.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
11.25
